# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Part II. Working with Apache Cassandra.

## Work with csv file named <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Step 1: Staging

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS nosqlproject
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('nosqlproject')
except Exception as e:
    print(e)

## Step 2: Working with data!

### Mission 1: 
#### Find out the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Solve:
##### Name table as ```session_songs```
##### Why partition key is ```sessionId``` and clustering key is ```itemInSession```? <br>-> Our table is ```session_songs``` having detailed info about session in each row with each particular session, each session has many ```itemInSession``` values. So I want to filter a specific ```itemInSession``` using where clause.

In [9]:
query = """
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY(sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Ingest data from csv into created new table.

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = """SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4"""
rows = session.execute(query)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Mission 2: 
#### Find out the only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Solve
##### Name table as ```user_songs```
##### Why partition key is ```(userId,sessionId)``` and clustering key is ```itemInSession```? <br>-> The use of ```(userId,sessionId)``` as partition key will pass the uniquely identify each row in the table and solve the performance issue when the volume of dataset is large as well as satisfy the filtered requirement by making ```(itemInSession)``` as the clustering key.

In [12]:
query = """
    CREATE TABLE IF NOT EXISTS user_songs
    (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Ingest data from csv into created new table.

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song, firstName, lastName))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
query = """SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182"""
rows = session.execute(query)

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Mission 3: 
#### Find out every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Solve
##### Name table as ```app_history```
##### Why partition key is ```song``` and clustering key is ```userId```? <br>-> A specific song can be found by making the ```song``` as the partition key and using where clause. First and Last name are not unique metrics, so that ```userId``` should be made as the clustering key to get the unique data. 

In [16]:
query = """
    CREATE TABLE IF NOT EXISTS app_history
    (song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Ingest data from csv into created new table.

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, int(userId), firstName, lastName))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
query = """SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'"""
rows = session.execute(query)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [19]:
app_history_dropquery = """DROP TABLE app_history"""
user_songs_dropquery = """DROP TABLE user_songs"""
session_songs_dropquery = """DROP TABLE session_songs"""
session.execute(app_history_dropquery)
session.execute(user_songs_dropquery)
session.execute(session_songs_dropquery)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()